# Mekong River Study Area

This is the annotated version of the notebook created by Sven-Arne Quist to try the CoastSat algorithm on a study area in the Mekong River. 
More background on the study area can be found here: https://www.nature.com/articles/s41598-019-53804-z. In addition, we have bathymetry data available for this research area, which we can use during the development of this product. 

This notebook is created for reading purposes. **You don't need to execute this notebook** (unless you have 8 hours spare time). 
> The notebook contains direct quotes from an example notebook provided in the CoastSat repo: https://github.com/kvos/CoastSat/blob/master/example_jupyter.ipynb.
> The original algorithm was published in: 
>> Vos K., Splinter K.D., Harley M.D., Simmons J.A., Turner I.L. (2019). CoastSat: a Google Earth Engine-enabled Python toolkit to extract shorelines from publicly available satellite imagery. Environmental Modelling and Software. 122, 104528. https://doi.org/10.1016/j.envsoft.2019.104528

My comments are provided in normal text. 
Ideas about **tweaks** to add at a particular place in the model are highligthed in bold 

> ## Initial settings
> 
> Refer to the **Installation** section of the README for instructions on how to install the Python packages necessary to run the software, including Google Earth Engine Python API. If that step has been completed correctly, the following packages should be imported without any problem.

I added geopandas in there for the compatibility with polygon files to define the area of interest. 

In [2]:
import os
import numpy as np
import pickle
import warnings
import geopandas as gpd
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
from coastsat import SDS_download, SDS_preprocess, SDS_shoreline, SDS_tools, SDS_transects

Load area of interest into "coords". This is a change made by me to allow for uploading of geojson files to the algorithm. The code below unpacks the coordinates into a list of lists called "coords". 

**Possible tweaks**: 
1. Allowing for kml files that can be user defined in Google Earth and read into the algorithm. 
2. Or adding an interactive webmapping tool that allows the user to draw polygons of the area of interest and automatically upload them to the algorithm. 

In [3]:
aoi_poly = gpd.read_file("Mekong_StudyArea.geojson")
g = [i for i in aoi_poly.geometry]
x,y = g[0].exterior.coords.xy
coords = np.dstack((x,y)).tolist()
print(coords)

[[[105.7683639, 10.3287573], [105.7676773, 10.2517387], [105.9413986, 10.2497117], [105.9393386, 10.3250419], [105.7683639, 10.3287573]]]


# 1. Retrieval of the images from GEE


>Define the region of interest (`polygon`), the date range (`dates`) and the satellite missions (`sat_list`) from which you wish to retrieve the satellite images. The images will be cropped on the Google Earth Engine server and only the region of interest will be downloaded as a .tif file. The files will stored in the directory defined in `filepath`. 
>
>Make sure the area of your ROI is smaller than 100 km2 (if larger split it into smaller ROIs).

Here the parameters are filled in by me already for the Mekong River example.  

**Possible tweaks**: within the "dates" variable, the upper bound of the time series (most recent date) can be automatically updated by the system date obtained from the computer. 

In [17]:
# region of interest (longitude, latitude)

polygon = coords 
# date range
dates = ['1985-01-01', '2020-04-01']
# satellite missions
sat_list = ['L5', 'L7', 'L8', 'S2']
# name of the site
sitename = 'MEKONG'
# directory where the data will be stored
filepath = os.path.join(os.getcwd(), 'data')
# put all the inputs into a dictionnary
inputs = {'polygon': polygon, 'dates': dates, 'sat_list': sat_list, 'sitename': sitename, 'filepath':filepath}


>The function `SDS_download.check_images_available(inputs)` will print the number of images available for your inputs. The Landsat images are divided in Tier 1 and Tier 2, only Tier 1 images can be used for time-series analysis. 

No change needed here

In [18]:
# before downloading the images, check how many images are available for your inputs
SDS_download.check_images_available(inputs);

Images available between 1985-01-01 and 2020-04-01:
- In Landsat Tier 1 & Sentinel-2 Level-1C:
  L5: 223 images
  L7: 232 images
  L8: 126 images
  S2: 525 images
  Total: 1106 images
- In Landsat Tier 2:
  L5: 132 images
  L7: 63 images
  L8: 26 images
  Total: 221 images



> "The function `SDS_download.retrieve_images(inputs)` retrives the satellite images from Google Earth Engine.
>
> By default, only Landsat Tier 1 and Sentinel-2 Level-1C are downloaded. 
>
> In case you need to access Tier 2 images for qualitative analysis, you need to set `inputs['include_T2'] = True` before calling `retrieve_images`."

To get an idea of how long it took to download the image library defined above: it took 3 hours. I'm not sure whether we need Tier-2 images as well, because it will only increase the download and analysis time. 

In [ ]:
# inputs['include_T2'] = True
metadata = SDS_download.retrieve_images(inputs)

Images available between 1985-01-01 and 2020-04-01:
- In Landsat Tier 1 & Sentinel-2 Level-1C:
  L5: 223 images
  L7: 232 images
  L8: 126 images
  S2: 525 images
  Total: 1106 images
- In Landsat Tier 2:
  L5: 132 images
  L7: 63 images
  L8: 26 images
  Total: 221 images

L5: 223 images
3%

> " **If you have already retrieved the images**, just load the metadata file by only running the section below"

In [4]:
metadata = SDS_download.get_metadata(inputs) 

# 2. Shoreline Extraction

> This section maps the position of the shoreline on the satellite images. The user can define the cloud threhold (`cloud_thresh`) and select the spatial reference system in which to output the coordinates of the mapped shorelines (`output_epsg`). See http://spatialreference.org/ to find the EPSG number corresponding to your local coordinate system. Make sure that your are using cartesian coordinates and not spherical coordinates (lat,lon) like WGS84. 
>
>To quality control each shoreline detection and manually validate the mapped shorelines, the user has the option to set the parameter `check_detection` to **True**. To save a figure for each mapped shoreline set `save_figure` to **True**. 
>
>The other parameters are for advanced users only and are described in the README.

- **Cloud_thresh** Here we might tune up the cloud threshold to reduce the data volume and get possibly get a more accurate database. However, this might come at the expense of the temporal dimension of the time series. 
- **check_detection** This should always set to False, otherwise you are forced to manually judge every image in the time series.
- **save_figure**: **tweak needed here** Currently, the model saves an image of the classification output (with classes of 'water', 'sand', and 'whitewater') and the shoreline on the model. We should adjust the model in such a way that it discards extracting the shoreline alltogether, and in stead just saves the output of the classified images. 
- advanced settings: I downtuned the 'min_beach_area' by a factor of 10.  
  **Tweak needed here**: I think we might discard the beach detection part of the algorithm all togehter, or add a functionality were we can specifiy whether the river bank is sandy or not. This boolean will then activate the sand detection algorithm, and take up analysis from there according to the regular CoastSat proceedure. 

In [5]:
settings = { 
    # general parameters:
    'cloud_thresh': 0.5,        # threshold on maximum cloud cover
    'output_epsg': 28356,       # epsg code of spatial reference system desired for the output   
    # quality control:
    'check_detection': False,    # if True, shows each shoreline detection to the user for validation
    'save_figure': True,        # if True, saves a figure showing the mapped shoreline for each image
    'color_style': False,       # if True, saves figure as true color image. If False, saves figure as false color image. 
    # add the inputs defined previously
    'inputs': inputs,
    # [ONLY FOR ADVANCED USERS] shoreline detection parameters:
    'min_beach_area': 450,     # minimum area (in metres^2) for an object to be labelled as a beach
    'buffer_size': 150,         # radius (in metres) of the buffer around sandy pixels considered in the shoreline detection
    'min_length_sl': 200,       # minimum length (in metres) of shoreline perimeter to be valid
    'cloud_mask_issue': False,  # switch this parameter to True if sand pixels are masked (in black) on many images  
    'sand_color': 'default',    # 'default', 'dark' (for grey/black sand beaches) or 'bright' (for white sand beaches)
}

> ### [OPTIONAL] Save .jpg of the satellite images 
> Saves .jpg files of the preprocessed satellite images (cloud masking + pansharpening/down-sampling) under *./data/sitename/jpeg_files\preprocessed*

**Tweak** I don't think we need this part for the final product, it just adds unnecessary data and time in the analysis. But it is handy to look at during the development phase 

In [11]:
# save jpeg of processed satellite image
SDS_preprocess.save_jpg(metadata, settings)

Satellite images saved as .jpg in C:\Users\Administrator\Downloads\CoastSat-master\CoastSat-master\data\MEKONG\jpg_files\preprocessed


>### [OPTIONAL] Digitize a reference shoreline
Creates a reference shoreline which helps to identify outliers and false detections. The reference shoreline is manually digitised by the user on one of the images. The parameter `max_dist_ref` defines the maximum distance from the reference shoreline (in metres) at which a valid detected shoreline can be. If you think that the default value of 100 m will not capture the full shoreline variability of your site, increase this value to an appropriate distance.

Here I digitzed the northern river bank to help the algorithm on its way. 
We may or may not need this proceedure depending on whether the riverbank is sandy or not. In the end, the analysis will look completely different for non-sandy river banks, so then this feature will not be necessary. 

In [6]:
%matplotlib qt
settings['reference_shoreline'] = SDS_preprocess.get_reference_sl(metadata, settings)
settings['max_dist_ref'] = 100 # max distance (in meters) allowed from the reference shoreline

Reference shoreline already exists and was loaded


>### Batch shoreline detection
>Extracts the 2D shorelines from the images in the spatial reference system specified by the user in `'output_epsg'`. The mapped shorelines are saved into `output.pkl` (under *./data/sitename*) and `output.geojson` (to be used in a GIS software).
>
>If you see that the sand pixels on the images are not being identified, change the parameter `sand_color` from `default` to `dark` or `bright` depending on the color of your beach. 

To get an idea of how long this takes: it took me 5 and a half hours to run this. This is by far the most heavy processing part of the algorithm, but we may not use it in the end for non-sandy river banks. In stead we should develop something else (to be discussed)

In [7]:
#moment of truth
%matplotlib qt
output = SDS_shoreline.extract_shorelines(metadata, settings)

Mapping shorelines:
L5:   100%
L7:   100%
L8:   100%
S2:   75%Could not map shoreline for this image: 2019-05-21-03-35-15_S2_MEKONG_10m.tif
S2:   100%


> Simple plot of the mapped shorelines. The coordinates are stored in the output dictionnary together with the exact dates in UTC time, the georeferencing accuracy and the cloud cover.

In this plot you can see all the shorelines as a shape in a graph. In this way you can compare them against each other and spot outliers. I did not find this plot very useful. 

In [8]:
fig = plt.figure()
plt.axis('equal')
plt.xlabel('Eastings')
plt.ylabel('Northings')
plt.grid(linestyle=':', color='0.5')
for i in range(len(output['shorelines'])):
    sl = output['shorelines'][i]
    date = output['dates'][i]
    plt.plot(sl[:,0], sl[:,1], '.', label=date.strftime('%d-%m-%Y'))
plt.legend()
mng = plt.get_current_fig_manager()                                         
mng.window.showMaximized()    
fig.set_size_inches([15.76,  8.52])

> ## 3. Shoreline analysis
>
> In this section we show how to compute time-series of cross-shore distance along user-defined shore-normal transects.
>
> **If you have already mapped the shorelines**, just load the output file (`output.pkl`) by running the section below

**Important:** 
Here is the most promising section of the code. Once we classified the river on every image after the download phase, we can measure the width of the river in a time series. The principle highligthed over here is very similar, although you look at the coastline change in this example. 

**Tweak** We should develop an automated proceedure to make cross sections for the river that can be analyzed by this time series method. 

In [9]:
filepath = os.path.join(inputs['filepath'], sitename)
with open(os.path.join(filepath, sitename + '_output' + '.pkl'), 'rb') as f:
    output = pickle.load(f)

>There are 3 options to define the coordinates of the shore-normal transects:
>
> **Option 1**: the user can interactively draw the shore-normal transects along the beach by calling:

I implemented option one over here because it was quick and easy. We should develop a better method to place transects across the river to measure the width. 

In [14]:
%matplotlib qt
transects = SDS_transects.draw_transects(output, settings)

Transect locations saved in C:\Users\Administrator\Downloads\CoastSat-master\CoastSat-master\data\MEKONG


>**Option 2**: the user can load the transect coordinates (make sure the spatial reference system is the same as defined previously by the parameter *output_epsg*) from a .geojson file by calling:

Just listed here for completeness, not implemented here: 

In [ ]:
geojson_file = os.path.join(os.getcwd(), 'examples', 'NARRA_transects.geojson')
transects = SDS_tools.transects_from_geojson(geojson_file)

>**Option 3**: manually provide the coordinates of the transects as shown in the example below:

Probably in the direction of the most optimal way to spread the transects over the river area. Maybe we can automatically spread the crossections over the area with defined regular intervals. Again, not implemented here but listed for completeness. 

In [ ]:
transects = dict([])
transects['Transect 1'] = np.array([[342836, 6269215], [343315, 6269071]])
transects['Transect 2'] = np.array([[342482, 6268466], [342958, 6268310]])
transects['Transect 3'] = np.array([[342185, 6267650], [342685, 6267641]])

> Now, intersect the transects with the 2D shorelines to obtain time-series of cross-shore distance.
>
>The time-series of shoreline change for each transect are saved in a .csv file in the data folder (all dates are in UTC time). 


In [15]:
# defines the along-shore distance over which to consider shoreline points to compute the median intersection (robust to outliers)
settings['along_dist'] = 25 
cross_distance = SDS_transects.compute_intersection(output, transects, settings) 

Time-series of the shoreline change along the transects saved as:
C:\Users\Administrator\Downloads\CoastSat-master\CoastSat-master\data\MEKONG\transect_time_series.csv


> Plot the time-series of shoreline change along each transect

We can keep the plot settings for a great deal, since it is usefull. 

**Tweak:** expand the timeseries with the BFAST algorithm (unfortunately only available in R), to study the time series and the signal decomposition.  

In [16]:
from matplotlib import gridspec
import numpy as np
fig = plt.figure()
gs = gridspec.GridSpec(len(cross_distance),1)
gs.update(left=0.05, right=0.95, bottom=0.05, top=0.95, hspace=0.05)
for i,key in enumerate(cross_distance.keys()):
    if np.all(np.isnan(cross_distance[key])):
        continue
    ax = fig.add_subplot(gs[i,0])
    ax.grid(linestyle=':', color='0.5')
    ax.set_ylim([-50,50])
    ax.plot(output['dates'], cross_distance[key]- np.nanmedian(cross_distance[key]), '-^', markersize=6)
    ax.set_ylabel('distance [m]', fontsize=12)
    ax.text(0.5,0.95, key, bbox=dict(boxstyle="square", ec='k',fc='w'), ha='center',
            va='top', transform=ax.transAxes, fontsize=14)
mng = plt.get_current_fig_manager()                                         
mng.window.showMaximized()    
fig.set_size_inches([15.76,  8.52])